In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd

In [2]:
LETRAS_ROOT_URL = "https://www.letras.com"
LETRAS_REGGAETON_URL = LETRAS_ROOT_URL + "/mais-acessadas/reggaeton/"

response = requests.get(LETRAS_REGGAETON_URL)

soup = BeautifulSoup(response.text, "html.parser")

# Achicamos la búsqueda al contenedor
artists_container = soup.find("ol", {'class': 'top-list_art'})

In [3]:
# Como ya hemos achicado al contenedor, buscamos el patrón que se repite. En este caso las etiquetas "a"
artists_html = artists_container.find_all("a")

In [4]:
# Creamos la lista con las URL de los artistas. Esta lista está compuesta por listas de 2 elementos: URL y nombre del artista.
artists_href = [[artist.get('href'), artist.text] for artist in artists_html]

# Imprimimos la lista.
# pprint(artists_href)

# Creamos un dataframe con las columnas artista, título de la canción y la letra.
df = pd.DataFrame(columns=['Artist', 'Title', 'Lyrics'])

In [5]:
for artist_href in artists_href:
    artist_href[0] = LETRAS_ROOT_URL + artist_href[0]

    art_response = requests.get(artist_href[0])
    art_soup = BeautifulSoup(art_response.text, "html.parser")

    # Achicamos la búsqueda al contenedor
    songs_container = art_soup.find("ol", {'class': 'cnt-list cnt-list--num cnt-list--col2'})
    
    # Como ya hemos achicado al contenedor, buscamos el patrón que se repite. En este caso es la etiqueta "a".
    songs_html = songs_container.find_all("a")

    # Creamos la lista con las URL de las canciones. Esta lista está compuesta por listas de 2 elementos: URL y título de la canción.
    songs_href = [[song.get('href'), song.text] for song in songs_html]

    # Imprimimos las URL de cada una de las canciones de los artistas.
    # pprint(songs_href)

    for song_href in songs_href:
        song_href[0] = LETRAS_ROOT_URL + song_href[0]
        
        song_response = requests.get(song_href[0])
        song_soup = BeautifulSoup(song_response.text, "html.parser")

        # Achicamos la búsqueda al contenedor
        paragraphs_container = song_soup.find("div", {'class': 'cnt-letra p402_premium'})
        #lyrics = song_soup.find("div", {'class': 'cnt-letra p402_premium'}).text

        # Como ya hemos achicado al contenedor, buscamos el patrón que se repite. En este caso las etiquetas "p".
        paragraphs_html = paragraphs_container.find_all("p")
        
        # Creamos la lista con las letras de las canciones.
        lyrics = [paragraph.text for paragraph in paragraphs_html]

        # Imprimimos las URL de cada una de las canciones de los artistas.
        # pprint(lyrics)
        
        # Vamos añadiendo al dataframe cada letra, con su título y artista correspondiente.
        # Incluyo "ignore_index=True" para que los index se vayan añadiendo de forma incremental.
        df = df.append({'Artist': artist_href[1], 'Title': song_href[1], 'Lyrics': lyrics} , ignore_index=True)


KeyboardInterrupt: 

In [6]:
df.head(50)

,Artist,Title,Lyrics
0,Karol G,Ocean,[Si algún día te vas de casa (de casa)Yo te ll...
1,Karol G,Dices Que Te Vas (part. Anuel AA),"[No sé, no-no séYo me despierto y busco tus me..."
2,Karol G,Love With a Quality (feat. Damian Marley),"[Chao, papaNow greetings to the worldVoice of ..."
3,Karol G,Punto G,"[(Dale, ey, ey), Sigue en la pista, me vas a e..."
4,Karol G,Créeme (part. Maluma),"[YayYaDice, ¿Cómo explicarle a la conciencia q..."
5,Karol G,Culpables (part. Anuel AA),"[Uah, uahMera, dime Karol G, bebesitaAnuel (oh..."
6,Karol G,A Ella,[Ella jugó a darte lo que más queríasYo jugué ...
7,Karol G,Mi Cama,[Dices que de mí ya te olvidasteY de tu mente ...
8,Karol G,Sin Corazón,[Yo no creo en el amor y tú no crees en el amo...
9,Karol G,Go Karo,"[O-O-Ovy On The Drums, Go, Karo!Enloqueció la ..."


In [10]:
df['Lyrics'][0]

['Si algún día te vas de casa (de casa)Yo te llevo a la NASAYeh, pido un cohete y voy directo por tiY si no estoy y algo te pasaRecuerda que todo en la vida cambiaY no importa lo que pase, te prometo no faltarte',
 'Me siento grande por tiY aunque lo intentara, no podría sin tiToda mi felicidad es gracias a tiY si yo me muero, volvería por ti',
 'Me siento grande por tiY aunque lo intentara, no podría sin tiToda mi felicidad es gracias a tiY si yo me muero, volvería por ti, por ti',
 'Se quedan cortas las palabras realmenteY a Dios le doy gracias porque estás aquí presenteQuiero que sepas que te amo eternamenteY que cuando dije sí, lo dije para siempre',
 "A tu lado todo no es perfecto, pero sí mejorY cada detalle tuyo e' mejor que el anteriorAquella canciónY cuando decoras con rosas mi habitación",
 "Vamos a enseñarle al mundo lo que es amorTú y yo podemos juntos, ehPorque amo toda' las locuras de tu menteY así me encanta presumirte ante la gente",
 'Me siento grande por tiY aunque lo

In [7]:
df['Lyrics'][0]

['Yo te llevo a la NASAYeh, pido un cohete y voy directo por tiY si no estoy y algo te pasaRecuerda que todo en la vida cambiaY no importa lo que pase, te prometo no faltarte',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [7]:
df['Lyrics'][0]

" Si algún día te vas de casa (de casa)Yo te llevo a la NASAYeh, pido un cohete y voy directo por tiY si no estoy y algo te pasaRecuerda que todo en la vida cambiaY no importa lo que pase, te prometo no faltarteMe siento grande por tiY aunque lo intentara, no podría sin tiToda mi felicidad es gracias a tiY si yo me muero, volvería por tiMe siento grande por tiY aunque lo intentara, no podría sin tiToda mi felicidad es gracias a tiY si yo me muero, volvería por ti, por tiSe quedan cortas las palabras realmenteY a Dios le doy gracias porque estás aquí presenteQuiero que sepas que te amo eternamenteY que cuando dije sí, lo dije para siempreA tu lado todo no es perfecto, pero sí mejorY cada detalle tuyo e' mejor que el anteriorAquella canciónY cuando decoras con rosas mi habitaciónVamos a enseñarle al mundo lo que es amorTú y yo podemos juntos, ehPorque amo toda' las locuras de tu menteY así me encanta presumirte ante la genteMe siento grande por tiY aunque lo intentara, no podría sin tiTo

In [7]:
df['Lyrics'][0]

['Yo te llevo a la NASAYeh, pido un cohete y voy directo por tiY si no estoy y algo te pasaRecuerda que todo en la vida cambiaY no importa lo que pase, te prometo no faltarte',
 '',
 '',
 '',
 '']